In [31]:
!pip install requests
!pip install tmdbsimple

In [32]:
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
import os, time, json, requests, glob
import numpy as np
import pandas as pd

In [33]:
# create a folder for saving files (if it doesn't ecist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['basics.csv',
 'movies.csv',
 'ratings.csv',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2010.json']

In [47]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename).
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data)==list) and (type(file_data == list)):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Set file's current position at offset.
        file.seek(0)
        # Convert back to json.
        json.dump(file_data, file)

In [35]:
#Load API creadentials
with open('/Users/HP/.secret/tmdb_api.json') as f:
    login = json.load(f)

# Display keys of loaded file
login.keys()

dict_keys(['api_key', 'api_read_access_token'])

In [36]:
# Set API_KEY
tmdb.API_KEY = login['api_key']

In [37]:
# Define function to extract info, and add cretification to the info dict else return an error
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [38]:
df_title_basics = pd.read_csv("Data/basics.csv",sep=',',low_memory=False,on_bad_lines='skip',encoding='utf-8')
df_title_basics.info()
df_title_basics.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86979 entries, 0 to 86978
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86978 non-null  object 
 3   originalTitle   86978 non-null  object 
 4   isAdult         86979 non-null  int64  
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  86979 non-null  int64  
 8   genres          86979 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 6.0+ MB


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [39]:
YEARS_TO_GET = [2001,2002]
errors=[]

In [56]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # check if the file already exists
    file_exists = os.path.isfile(JSON_FILE)
    # if JSON_FILE does not exist: create it
    if file_exists == False:
        # Print a message indicating the file is being created
        print(f"Creating {JSON_FILE}) for API results for {YEAR}")
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)
    else:
        print(f"The file {JSON_FILE} already exists")
    # saving new year as current df
    df = df_title_basics.loc[df_title_basics['startYear']==YEAR].copy()
    # saving movie ids to a seperate variable
    movie_ids = df['tconst']
    # load existing data from json into dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    print(f"Number of Already Retrieved movie ids relative to year {YEAR} = {len(previous_df)}")
    print(f"Number of Remaining movie ids to GET relative to year {YEAR} = {len(movie_ids_to_get)}")
    # loop through movie_ids_to_get with a tqdm progress bar
    for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
        # Get index and movie id from list
        try:
            temp = get_movie_with_rating(movie_id)
            # append/extend results to existing file using the pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
        except Exception as e:
            errors.append([movie_id, e, YEAR])
    print(f"- Total errors of {YEAR}: {len(errors)}")
    #Save the year's results as csv.gz file
    try:
        final_year_df = pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}_{len(movie_ids_to_get)}.csv.gz", compression="gzip", index=False)
    except:
        pass

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

The file Data/tmdb_api_results_2001.json already exists
Number of Already Retrieved movie ids relative to year 2001 = 1367
Number of Remaining movie ids to GET relative to year 2001 = 210


Movies from 2001:   0%|          | 0/210 [00:00<?, ?it/s]

- Total errors of 2001: 914
The file Data/tmdb_api_results_2002.json already exists
Number of Already Retrieved movie ids relative to year 2002 = 527
Number of Remaining movie ids to GET relative to year 2002 = 1046


Movies from 2002:   0%|          | 0/1046 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Use glob to get all filepaths that match the pattern (*=wildcard)
tmdb_files = sorted(glob.glob("Data/final_tmdb_data*.csv.gz"))
tmdb_files


In [ ]:
import glob
# Use glob to get all filepaths that match the pattern (*=wildcard)
tmdb_files = sorted(glob.glob("Data/final_tmdb_data*.csv.gz"))
tmdb_files

In [ ]:
"""

# Adding lineterminator arg to get around error
df = pd.concat([pd.read_csv(f, lineterminator='\n') for f in files] )
